In [7]:
import pandas as pd
import requests

# === CONFIGURATION ===
CSV_FILE = "/Users/dequeue/Desktop/RUI.nosync/hra-registrations/staging/hubmap-fallopian_tube_millitome-fisher-2024/resources/PNNL_Lookup.csv"
API_TOKEN = "Ag3XePE7deB9kaX2Pkr39jeoBoby50wVeGwk2x9O1eDvqevP94cVCqD8Q4Mrg5x58OnMl14Bxg2Ob8fdEPPQbSQw8O4"
ENTITY_API = "https://entity.api.hubmapconsortium.org/entities/{}"
HEADERS = {"Authorization": f"Bearer {API_TOKEN}"}

# === HELPER FUNCTION ===
def get_uuid(hubmap_id):
    try:
        response = requests.get(ENTITY_API.format(hubmap_id), headers=HEADERS)
        if response.status_code == 200:
            data = response.json()
            return data.get("uuid")
        else:
            print(f"Failed for {hubmap_id}: {response.status_code}")
    except Exception as e:
        print(f"Error for {hubmap_id}: {e}")
    return None

# === LOAD DATA ===
df = pd.read_csv(CSV_FILE)

# Remove prefix from 'Donor' column
df["Donor"] = df["Donor"].str.replace("https://portal.hubmapconsortium.org/browse/", "", regex=False)

# Get UUIDs and build links
df["Donor_uuid"] = df["Donor"].apply(get_uuid)
df["Donor link"] = "https://portal.hubmapconsortium.org/browse/" + df["Donor_uuid"].fillna("")

df["Sample_uuid"] = df["HuBMAP Sample ID"].apply(get_uuid)
df["ID"] = "https://entity.api.hubmapconsortium.org/entities/" + df["Sample_uuid"].fillna("")

# === SAVE UPDATED FILE ===
output_file = "/Users/dequeue/Desktop/RUI.nosync/hra-registrations/staging/hubmap-fallopian_tube_millitome-fisher-2024/resources/PNNL_Lookup_with_UUIDs.csv"
df.to_csv(output_file, index=False)
print(f"✅ Updated CSV saved to: {output_file}")

Failed for nan: 400
Failed for nan: 400
Failed for nan: 400
Failed for nan: 400
Failed for nan: 400
Failed for nan: 400
Failed for nan: 400
Failed for nan: 400
✅ Updated CSV saved to: /Users/dequeue/Desktop/RUI.nosync/hra-registrations/staging/hubmap-fallopian_tube_millitome-fisher-2024/resources/PNNL_Lookup_with_UUIDs.csv


In [9]:
import pandas as pd
import requests

# === CONFIGURATION ===
CSV_FILE = "/Users/dequeue/Desktop/RUI.nosync/hra-registrations/staging/hubmap-fallopian_tube_millitome-fisher-2024/resources/PNNL_Lookup.csv"
API_TOKEN = "Ag3XePE7deB9kaX2Pkr39jeoBoby50wVeGwk2x9O1eDvqevP94cVCqD8Q4Mrg5x58OnMl14Bxg2Ob8fdEPPQbSQw8O4"
ENTITY_API = "https://entity.api.hubmapconsortium.org/entities/{}"
HEADERS = {"Authorization": f"Bearer {API_TOKEN}"}

def extract_donor_info(donor_id):
    url = ENTITY_API.format(donor_id)
    try:
        r = requests.get(url, headers=HEADERS)
        if r.status_code != 200:
            print(f"Failed for {donor_id}: {r.status_code}")
            return pd.Series([None, None, None])
        data = r.json()
        bmi = age = sex = None
        for entry in data.get("metadata", {}).get("organ_donor_data", []):
            if entry.get("preferred_term") == "Body Mass Index":
                bmi = entry.get("data_value")
            elif entry.get("preferred_term") == "Age":
                age = entry.get("data_value")
            elif entry.get("grouping_concept_preferred_term") == "Sex":
                sex = entry.get("data_value")
        return pd.Series([bmi, age, sex])
    except Exception as e:
        print(f"Error for {donor_id}: {e}")
        return pd.Series([None, None, None])

# === LOAD DATA ===
df = pd.read_csv(CSV_FILE)

# Remove prefix from 'Donor' column if present
df["Donor"] = df["Donor"].str.replace("https://portal.hubmapconsortium.org/browse/", "", regex=False)

# Add BMI, Age, Sex columns
df[["BMI", "Age", "Sex"]] = df["Donor"].apply(extract_donor_info)

# === SAVE UPDATED FILE ===
output_file = CSV_FILE.replace(".csv", "_with_BMI_Age_Sex.csv")
df.to_csv(output_file, index=False)
print(f"✅ Updated CSV saved to: {output_file}")

✅ Updated CSV saved to: /Users/dequeue/Desktop/RUI.nosync/hra-registrations/staging/hubmap-fallopian_tube_millitome-fisher-2024/resources/PNNL_Lookup_with_BMI_Age_Sex.csv


In [13]:
import pandas as pd

# === CONFIGURATION ===
CSV_FILE = "/Users/dequeue/Desktop/RUI.nosync/hra-registrations/staging/hubmap-fallopian_tube_millitome-fisher-2024/resources/PNNL_Lookup.csv"

# === LOAD DATA ===
df = pd.read_csv(CSV_FILE)

# === EXPAND BLOCK Location ===
expanded_rows = []
for _, row in df.iterrows():
    block_locs = str(row['BLOCK Location']).split(',')
    block_locs = [loc.strip() for loc in block_locs if loc.strip()]
    if len(block_locs) > 1:
        for loc in block_locs:
            new_row = row.copy()
            new_row['BLOCK Location'] = loc
            expanded_rows.append(new_row)
    else:
        expanded_rows.append(row)

expanded_df = pd.DataFrame(expanded_rows)

# === SAVE UPDATED FILE ===
output_file = CSV_FILE.replace(".csv", "_expanded.csv")
expanded_df.to_csv(output_file, index=False)
print(f"✅ Expanded CSV saved to: {output_file}")

✅ Expanded CSV saved to: /Users/dequeue/Desktop/RUI.nosync/hra-registrations/staging/hubmap-fallopian_tube_millitome-fisher-2024/resources/PNNL_Lookup_expanded.csv


In [17]:
import pandas as pd

# === CONFIGURATION ===
CSV_FILE = "/Users/dequeue/Desktop/RUI.nosync/hra-registrations/staging/hubmap-fallopian_tube_millitome-fisher-2024/resources/PNNL_Lookup.csv"

# === LOAD DATA ===
df = pd.read_csv(CSV_FILE)

# === MAP FOR PARENT ORGAN TO BASE URL ===
organ_url_map = {
    "F. tube (L)": "https://purl.humanatlas.io/millitome/fallopian-tube-female-left-upenn/v1.0",
    "F. tube (R)": "https://purl.humanatlas.io/millitome/fallopian-tube-female-right-upenn/v1.0",
    "ovary (L)": "https://purl.humanatlas.io/millitome/ovary-female-left-upenn/v1.0",
    "ovary (R)": "https://purl.humanatlas.io/millitome/ovary-female-right-upenn/v1.0",
    "uterus": "https://purl.humanatlas.io/millitome/uterus-female-upenn/v1.0"
}

def make_rui_location(row):
    base = organ_url_map.get(str(row['Parent organ type']).strip(), "")
    block = str(row['BLOCK Location']).strip()
    if base and block:
        return f"{base}#{block}"
    return ""

df['rui_location'] = df.apply(make_rui_location, axis=1)

# === SAVE UPDATED FILE ===
output_file = CSV_FILE.replace(".csv", "_with_rui_location.csv")
df.to_csv(output_file, index=False)
print(f"✅ Updated CSV saved to: {output_file}")

✅ Updated CSV saved to: /Users/dequeue/Desktop/RUI.nosync/hra-registrations/staging/hubmap-fallopian_tube_millitome-fisher-2024/resources/PNNL_Lookup_with_rui_location.csv


In [21]:
import pandas as pd
import yaml

# === CONFIGURATION ===
CSV_FILE = "/Users/dequeue/Desktop/RUI.nosync/hra-registrations/staging/hubmap-fallopian_tube_millitome-fisher-2024/resources/PNNL_Lookup.csv"

# === LOAD DATA ===
df = pd.read_csv(CSV_FILE)
df.columns = df.columns.str.strip()  # Remove leading/trailing spaces from column names

# Map parent organ values to output YAML filenames and organ keys
organ_map = {
    "F. tube (L)": "fallopian_tube",
    "F. tube (R)": "fallopian_tube",
    "uterus": "uterus",
    "ovary (L)": "ovary",
    "ovary (R)": "ovary"
}

# Group rows by organ type
for organ_key, organ_label in {
    "fallopian_tube": ["F. tube (L)", "F. tube (R)"],
    "uterus": ["uterus"],
    "ovary": ["ovary (L)", "ovary (R)"]
}.items():
    organ_df = df[df["Parent organ type"].isin(organ_label)]
    if organ_df.empty:
        continue

    # Build YAML structure
    yaml_data = [{
        "consortium_name": "HuBMAP",
        "provider_name": "University of Pennsylvania",
        "provider_uuid": "18460CE4-4BB4-11F0-A2B2-20E2A8B75C6B",
        "defaults": {
            "id": "https://hubmapconsortium.github.io/hra-registrations/hubmap-fallopian_tube-millitome-fisher-2024/rui_locations.jsonld",
            "thumbnail": "assets/icons/ico-unknown.svg",
            "link": "http://example.com/user/link"
        },
        "donors": []
    }]

    for donor_id, donor_rows in organ_df.groupby("Donor"):
        donor_info = donor_rows.iloc[0]
        donor_entry = {
            "id": donor_id,
            "sex": donor_info.get("Sex", ""),
            "age": donor_info.get("Age", ""),
            "link": donor_info.get("Donor link", ""),
            "samples": []
        }
        for _, row in donor_rows.iterrows():
            sample = {
                "sample_type": "Tissue Block",
                "label": row.get("Lab ID", ""),
                "link": row.get("Link", ""),
                "id": row.get("ID", ""),
                "rui_location": row.get("rui_location", "")
            }
            donor_entry["samples"].append(sample)
        yaml_data[0]["donors"].append(donor_entry)

    # Write YAML file
    output_file = f"/Users/dequeue/Desktop/RUI.nosync/hra-registrations/staging/hubmap-fallopian_tube_millitome-fisher-2024/{organ_key}_registrations.yaml"
    with open(output_file, "w") as f:
        yaml.dump(yaml_data, f, sort_keys=False, default_flow_style=False)
    print(f"✅ Wrote {output_file}")

✅ Wrote /Users/dequeue/Desktop/RUI.nosync/hra-registrations/staging/hubmap-fallopian_tube_millitome-fisher-2024/fallopian_tube_registrations.yaml
✅ Wrote /Users/dequeue/Desktop/RUI.nosync/hra-registrations/staging/hubmap-fallopian_tube_millitome-fisher-2024/uterus_registrations.yaml
✅ Wrote /Users/dequeue/Desktop/RUI.nosync/hra-registrations/staging/hubmap-fallopian_tube_millitome-fisher-2024/ovary_registrations.yaml


✅ Wrote /Users/dequeue/Desktop/RUI.nosync/hra-registrations/staging/hubmap-fallopian_tube_millitome-fisher-2024/fallopian_tube_registrations.yaml
✅ Wrote /Users/dequeue/Desktop/RUI.nosync/hra-registrations/staging/hubmap-fallopian_tube_millitome-fisher-2024/uterus_registrations.yaml
✅ Wrote /Users/dequeue/Desktop/RUI.nosync/hra-registrations/staging/hubmap-fallopian_tube_millitome-fisher-2024/ovary_registrations.yaml


In [26]:
import pandas as pd
import yaml

# === CONFIGURATION ===
CSV_FILE = "/Users/dequeue/Desktop/RUI.nosync/hra-registrations/staging/hubmap-fallopian_tube_millitome-fisher-2024/resources/PNNL_Lookup.csv"

# === LOAD DATA ===
df = pd.read_csv(CSV_FILE)
df.columns = df.columns.str.strip()  # Remove leading/trailing spaces from column names

# Group rows by organ type
organ_groups = {
    "fallopian_tube": ["F. tube (L)", "F. tube (R)"],
    "uterus": ["uterus"],
    "ovary": ["ovary (L)", "ovary (R)"]
}

for organ_key, organ_labels in organ_groups.items():
    organ_df = df[df["Parent organ type"].isin(organ_labels)]
    if organ_df.empty:
        continue

    # Build YAML structure
    yaml_data = [{
        "consortium_name": "HuBMAP",
        "provider_name": "University of Pennsylvania",
        "provider_uuid": "18460CE4-4BB4-11F0-A2B2-20E2A8B75C6B",
        "defaults": {
            "id": "https://hubmapconsortium.github.io/hra-registrations/hubmap-fallopian_tube-millitome-fisher-2024/rui_locations.jsonld",
            "thumbnail": "assets/icons/ico-unknown.svg",
            "link": "http://example.com/user/link"
        },
        "donors": []
    }]

    for donor_id, donor_rows in organ_df.groupby("Donor"):
        donor_info = donor_rows.iloc[0]
        # Build donor_entry in the desired order
        donor_entry = {
            "id": donor_info.get("Donor link", ""),
            "sex": donor_info.get("Sex", ""),
            "link": donor_info.get("Donor link", "")
        }
        age_val = donor_info.get("Age", "")
        if pd.notnull(age_val) and str(age_val).strip() != "":
            try:
                age_val_clean = float(age_val)
                if age_val_clean.is_integer():
                    age_val_clean = int(age_val_clean)
            except Exception:
                age_val_clean = str(age_val)
            donor_entry["age"] = age_val_clean
        # Add samples last
        donor_entry["samples"] = []
        for _, row in donor_rows.iterrows():
            sample = {
                "sample_type": "Tissue Block",
                "label": row.get("Lab ID", ""),
                "link": row.get("Link", ""),
                "id": row.get("ID", ""),
                "rui_location": row.get("rui_location", "")
            }
            donor_entry["samples"].append(sample)
        yaml_data[0]["donors"].append(donor_entry)

    # Write YAML file
    yaml.Dumper.ignore_aliases = lambda *args : True
    output_file = f"/Users/dequeue/Desktop/RUI.nosync/hra-registrations/staging/hubmap-fallopian_tube_millitome-fisher-2024/{organ_key}_registrations.yaml"
    with open(output_file, "w") as f:
        yaml.dump(yaml_data, f, sort_keys=False, default_flow_style=False)
    print(f"✅ Wrote {output_file}")

✅ Wrote /Users/dequeue/Desktop/RUI.nosync/hra-registrations/staging/hubmap-fallopian_tube_millitome-fisher-2024/fallopian_tube_registrations.yaml
✅ Wrote /Users/dequeue/Desktop/RUI.nosync/hra-registrations/staging/hubmap-fallopian_tube_millitome-fisher-2024/uterus_registrations.yaml
✅ Wrote /Users/dequeue/Desktop/RUI.nosync/hra-registrations/staging/hubmap-fallopian_tube_millitome-fisher-2024/ovary_registrations.yaml


In [1]:
import pandas as pd
import requests

# === CONFIGURATION ===
EXCEL_FILE = "/Users/dequeue/Desktop/RUI.nosync/hra-registrations/staging/hubmap-pancreas_millitome-thompson-2024/resources/lookup.xlsx"
ENTITY_API = "https://entity.api.hubmapconsortium.org/entities/{}"
API_TOKEN = "Ag3XePE7deB9kaX2Pkr39jeoBoby50wVeGwk2x9O1eDvqevP94cVCqD8Q4Mrg5x58OnMl14Bxg2Ob8fdEPPQbSQw8O4"
HEADERS = {"Authorization": f"Bearer {API_TOKEN}"}

# === LOAD DATA ===
df = pd.read_excel(EXCEL_FILE)
df.columns = df.columns.str.strip()  # Clean up column names

# === HELPER FUNCTION ===
def get_uuid(sample_id):
    try:
        response = requests.get(ENTITY_API.format(sample_id), headers=HEADERS)
        if response.status_code == 200:
            data = response.json()
            return data.get("uuid")
        else:
            print(f"Failed for {sample_id}: {response.status_code}")
    except Exception as e:
        print(f"Error for {sample_id}: {e}")
    return ""

# === PROCESS SAMPLE IDs ===
uuid_list = [get_uuid(sid) for sid in df["Sample ID"]]

df["Sample Link"] = "https://portal.hubmapconsortium.org/browse/" + df["Sample ID"].astype(str)
df["Sample UUID"] = uuid_list
df["Sample ID"] = "https://entity.api.hubmapconsortium.org/entities/" + df["Sample UUID"].fillna("")

# === SAVE UPDATED FILE ===
output_file = EXCEL_FILE.replace(".xlsx", "_with_UUIDs.xlsx")
df.to_excel(output_file, index=False)
print(f"✅ Updated Excel saved to: {output_file}")

/Users/dequeue/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


✅ Updated Excel saved to: /Users/dequeue/Desktop/RUI.nosync/hra-registrations/staging/hubmap-pancreas_millitome-thompson-2024/resources/lookup_with_UUIDs.xlsx


In [31]:
import openpyxl
print(openpyxl.__version__)

3.1.5


In [4]:
import pandas as pd

# === CONFIGURATION ===
EXCEL_FILE = "/Users/dequeue/Desktop/RUI.nosync/hra-registrations/staging/hubmap-pancreas_millitome-thompson-2024/resources/lookup2.0.xlsx"

# === LOAD DATA ===
df = pd.read_excel(EXCEL_FILE)
df.columns = df.columns.str.strip()  # Clean up column names

def make_rui_location(row):
    sex = str(row["Sex"]).strip().lower()
    lab_id = str(row["Lab ID"]).strip()
    # Choose base URL based on sex
    if sex == "male":
        base = "https://purl.humanatlas.io/millitome/pancreas-male-uf/v1.0#"
    elif sex == "female":
        base = "https://purl.humanatlas.io/millitome/pancreas-female-uf/v1.0#"
    else:
        return ""
    # Get the part after the hyphen in Lab ID
    if "-" in lab_id:
        suffix = lab_id.split("-", 1)[1]
    else:
        suffix = lab_id
    return f"{base}{suffix}"

df["rui_location"] = df.apply(make_rui_location, axis=1)

# === SAVE UPDATED FILE ===
output_file = EXCEL_FILE.replace(".xlsx", "_with_rui_location.xlsx")
df.to_excel(output_file, index=False)
print(f"✅ Updated Excel saved to: {output_file}")

✅ Updated Excel saved to: /Users/dequeue/Desktop/RUI.nosync/hra-registrations/staging/hubmap-pancreas_millitome-thompson-2024/resources/lookup2.0_with_rui_location.xlsx


In [3]:
print(df.columns.tolist())
print(df[["Sex", "Lab ID"]].head(10))

['Sample ID', 'Lab ID', 'Donor ID', 'Donor Link', 'Sex', 'Age', 'BMI', 'Sample Link', 'Sample UUID', 'rui_location']
    Sex Lab ID
0  Male   P1-1
1  Male   P1-2
2  Male   P1-3
3  Male  P1-4A
4  Male  P1-4B
5  Male  P1-4C
6  Male  P1-4D
7  Male  P1-5A
8  Male  P1-5B
9  Male  P1-5D


In [11]:
import pandas as pd
import yaml

# === CONFIGURATION ===
EXCEL_FILE = "/Users/dequeue/Desktop/RUI.nosync/hra-registrations/staging/hubmap-pancreas_millitome-thompson-2024/resources/lookup2.0.xlsx"

# === LOAD DATA ===
df = pd.read_excel(EXCEL_FILE)
df.columns = df.columns.str.strip()  # Clean up column names

# === BUILD YAML STRUCTURE ===
yaml_data = [{
    "consortium_name": "HuBMAP",
    "provider_name": "University of Pennsylvania",
    "provider_uuid": "18460CE4-4BB4-11F0-A2B2-20E2A8B75C6B",
    "defaults": {
        "id": "https://hubmapconsortium.github.io/hra-registrations/hubmap-pancreas-millitome-thompson-2024/rui_locations.jsonld",
        "thumbnail": "assets/icons/ico-unknown.svg",
        "link": "http://example.com/user/link"
    },
    "donors": []
}]

for donor_id, donor_rows in df.groupby("Donor ID"):
    donor_info = donor_rows.iloc[0]
    # Convert age and BMI to Python types or skip if missing
    age_val = donor_info.get("Age", "")
    bmi_val = donor_info.get("BMI", "")
    try:
        age_val = int(age_val) if pd.notnull(age_val) and str(age_val).strip() != "" else ""
    except Exception:
        age_val = str(age_val)
    try:
        bmi_val = float(bmi_val) if pd.notnull(bmi_val) and str(bmi_val).strip() != "" else ""
        if bmi_val and bmi_val.is_integer():
            bmi_val = int(bmi_val)
    except Exception:
        bmi_val = str(bmi_val)

    donor_entry = {
        "id": donor_id,
        "sex": donor_info.get("Sex", ""),
        "age": age_val,
        "BMI": bmi_val,
        "link": donor_info.get("Donor Link", ""),
        "samples": []
    }
    for _, row in donor_rows.iterrows():
        sample = {
            "sample_type": "Tissue Block",
            "id": row.get("Sample ID", ""),
            "link": row.get("Sample Link", ""),
            "rui_location": row.get("rui_location", "")
        }
        donor_entry["samples"].append(sample)
    yaml_data[0]["donors"].append(donor_entry)

# === SAVE YAML FILE ===
output_file = EXCEL_FILE.replace(".xlsx", "_registrations.yaml")
yaml.Dumper.ignore_aliases = lambda *args : True
with open(output_file, "w") as f:
    yaml.dump(yaml_data, f, sort_keys=False, default_flow_style=False)
print(f"✅ Wrote {output_file}")

✅ Wrote /Users/dequeue/Desktop/RUI.nosync/hra-registrations/staging/hubmap-pancreas_millitome-thompson-2024/resources/lookup2.0_registrations.yaml


In [17]:
import pandas as pd
import yaml

# === CONFIGURATION ===
EXCEL_FILE = "/Users/dequeue/Desktop/RUI.nosync/hra-registrations/staging/hubmap-pancreas_millitome-thompson-2024/resources/lookup2.0.xlsx"

# === LOAD DATA ===
df = pd.read_excel(EXCEL_FILE)
df.columns = df.columns.str.strip()  # Clean up column names

# Normalize Donor ID for grouping
df["Donor ID"] = df["Donor ID"].astype(str).str.strip()

# === BUILD YAML STRUCTURE ===
yaml_data = [{
    "consortium_name": "HuBMAP",
    "provider_name": "University of Pennsylvania",
    "provider_uuid": "18460CE4-4BB4-11F0-A2B2-20E2A8B75C6B",
    "defaults": {
        "id": "https://hubmapconsortium.github.io/hra-registrations/hubmap-pancreas-millitome-thompson-2024/rui_locations.jsonld",
        "thumbnail": "assets/icons/ico-unknown.svg",
        "link": "http://example.com/user/link"
    },
    "donors": []
}]

for donor_id, donor_rows in df.groupby("Donor ID", sort=False):
    donor_info = donor_rows.iloc[0]
    # Convert age and BMI to Python types or skip if missing
    age_val = donor_info.get("Age", "")
    bmi_val = donor_info.get("BMI", "")
    try:
        age_val = int(age_val) if pd.notnull(age_val) and str(age_val).strip() != "" else ""
    except Exception:
        age_val = str(age_val)
    try:
        bmi_val = float(bmi_val) if pd.notnull(bmi_val) and str(bmi_val).strip() != "" else ""
        if bmi_val and bmi_val.is_integer():
            bmi_val = int(bmi_val)
    except Exception:
        bmi_val = str(bmi_val)

    donor_entry = {
        "id": donor_id,
        "sex": donor_info.get("Sex", ""),
        "age": age_val,
        "BMI": bmi_val,
        "link": donor_info.get("Donor Link", ""),
        "samples": []
    }
    for _, row in donor_rows.iterrows():
        sample = {
            "sample_type": "Tissue Block",
            "label": row.get("Lab ID", ""),
            "id": row.get("Sample ID", ""),
            "link": row.get("Sample Link", ""),
            "rui_location": row.get("rui_location", "")
        }
        donor_entry["samples"].append(sample)
    yaml_data[0]["donors"].append(donor_entry)

# === SAVE YAML FILE ===
output_file = EXCEL_FILE.replace(".xlsx", "_registrations.yaml")
yaml.Dumper.ignore_aliases = lambda *args : True
with open(output_file, "w") as f:
    yaml.dump(yaml_data, f, sort_keys=False, default_flow_style=False)
print(f"✅ Wrote {output_file}")

✅ Wrote /Users/dequeue/Desktop/RUI.nosync/hra-registrations/staging/hubmap-pancreas_millitome-thompson-2024/resources/lookup2.0_registrations.yaml
